In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('data-kmeans.csv')
data = dataset.values

In [ ]:
def compute_distance(a, b):

    dist = #distance between a and b#

    return dist

In [ ]:
def compute_centroid(Z):

    center = #centroid of a set of points in P#
    
    return center

In [ ]:
def compute_label(z, M):

    label = #label of point z with a set of centroids M#
    
    return label

In [ ]:
def compute_loss(C, M):

    loss = #compute loss#
    
    return loss